In [1]:
import open3d as o3d 
from visualization_utils import * 
import numpy as np 
from metrics.metrics_class import Metrics
from tqdm import tqdm 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]

def iou(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        union = np.union1d(pred_indices, gt_indices)
        return intersection.size / union.size

def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels
        
def get_semantics(preds,gt_idcs,pcd):
        new_ncuts_labels = np.zeros_like(preds)
        new_pcd = o3d.geometry.PointCloud()
        new_pcd.points = pcd.points 
        num_clusters = 0 
        for i in np.unique(preds):
                #new_cols = np.zeros((np.asarray(pcd.points).shape[0],3))

                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                
                #new_cols[pred_idcs] = np.array([1,0,0])
                #new_pcd.colors = o3d.utility.Vector3dVector(new_cols)
                #o3d.visualization.draw_geometries([new_pcd])

                if cur_intersect > 0.1:
                        #print("intersect",cur_intersect)
                        new_ncuts_labels[pred_idcs] = 1
                        num_clusters += 1
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd,new_ncuts_labels)])
        return np.where(new_ncuts_labels == 1)[0], num_clusters

from concurrent.futures import ThreadPoolExecutor

def process_batch(unique_pred, preds, labels, gt_idcs, threshold, new_ncuts_labels):
    pred_idcs = np.where(preds == unique_pred)[0]
    cur_intersect = np.sum(np.isin(pred_idcs, gt_idcs))
    if cur_intersect > threshold * len(pred_idcs): 
        new_ncuts_labels[pred_idcs] = 0

def remove_semantics(labels, preds, threshold=0.8, num_threads=8):
    gt_idcs = np.where(labels == 0)[0]
    new_ncuts_labels = preds.copy()
    unique_preds = np.unique(preds)
    
    if num_threads is None:
        num_threads = min(len(unique_preds), 8)  # Default to 8 threads if not specified
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for i in tqdm(unique_preds):
            futures.append(executor.submit(process_batch, i, preds, labels, gt_idcs, threshold, new_ncuts_labels))
        
        # Wait for all tasks to complete
        for future in tqdm(futures, total=len(futures), desc="Processing"):
            future.result()  # Get the result to catch any exceptions
        
    return new_ncuts_labels

In [3]:
def remove_cols(pcd,cols,labels,unique_labels,idx=1): 

	new_pcd = o3d.geometry.PointCloud()
	new_pcd.points = pcd.points 
	new_cols = np.zeros((np.asarray(pcd.points).shape[0],3))
	pcd_cols = np.asarray(pcd.colors)
	
	idcs = np.where(labels == unique_labels[idx])[0]
	
	
	new_cols[idcs] = cols[idx]
	new_pcd.colors = o3d.utility.Vector3dVector(new_cols)
	#o3d.visualization.draw_geometries([new_pcd])
	return new_pcd,idcs 
	
	
def cluster_vis(pcd,cols,labels,unique_labels,idx=1): 

	new_pcd = o3d.geometry.PointCloud()
	new_pcd.points = pcd.points 
	new_cols = np.zeros((np.asarray(pcd.points).shape[0],3))
	pcd_cols = np.asarray(pcd.colors)
	
	idcs = np.where(labels == unique_labels[idx])[0]
	
	
	new_cols[idcs] = cols[idx]
	new_pcd.colors = o3d.utility.Vector3dVector(new_cols)
	#o3d.visualization.draw_geometries([new_pcd])
	return new_pcd,idcs 
	
	
	

In [4]:
import os 

SEQUENCE_NUM = 7


out_folder = 'pcd_preprocessed/out_maps/' + str(SEQUENCE_NUM) + '/'
chunk_folder = 'pcd_preprocessed/output_chunks/'

fs = []

#fs.append(out_folder + 'hdbscan_all'  + str(SEQUENCE_NUM) + '.pcd')
#fs.append(out_folder + '3duis_' + str(SEQUENCE_NUM) + '.pcd')

'''
fs.append(out_folder + 'dinov2_only.pcd')
fs.append(out_folder + 'theta_0.5_alpha_1.0_t_0.03.pcd')
fs.append(out_folder + 'tarl_spatial_sam.pcd')
fs.append(out_folder + 'sam_0.5_spatial_2.0_t_0.03.pcd')
fs.append(out_folder + 'ncuts_sam.pcd')
fs.append(out_folder + 'ncuts_spatial1.pcd')
fs.append(out_folder + 'ncuts_spatial2.pcd')
'''
fs = os.listdir(out_folder)
fs = [out_folder + f for f in fs]


sem_gt = out_folder + "merge_part_kitti_semantic7.pcd"
inst_gt = out_folder + "merge_part_kitti_instance" + str(SEQUENCE_NUM) + ".pcd"

pcd_gt = o3d.io.read_point_cloud(inst_gt)
unique_colors, labels_kitti = np.unique(np.asarray(pcd_gt.colors),axis=0, return_inverse=True)

vis_fs = ['theta_0.5_alpha_1.0_t_0.03.pcd','merge_part_kitti_instance7.pcd','3duis_7.pcd','merge_refined7.pcd']

fs_gt = os.listdir(chunk_folder + 'out_kitti_semantic/')
for f in fs :
	#if f == inst_gt : 
	fn =   f.split('/')[-1]
	print(fn)
	if fn not in vis_fs:
		continue
	pred_pcd = o3d.io.read_point_cloud(f)
	unique_colors, labels_ncuts = np.unique(np.asarray(pred_pcd.colors), axis=0, return_inverse=True)
	print('map name',f)
	if fn != inst_gt:
		new_labels = remove_semantics(labels_kitti,labels_ncuts)
		o3d.visualization.draw_geometries([color_pcd_by_labels(pred_pcd,new_labels)])
	else : 
		o3d.visualization.draw_geometries([pred_pcd])
	'''
	metrics = Metrics(f)
	metrics.calc_ap = False
	metrics.calc_all = False
	
	metrics.update_stats(labels_ncuts,new_labels,labels_kitti)
	'''
	
	'''
	print("f",f)
	sem_gt = chunk_folder + 'out_kitti_semantic/'  + f.split('.')[0] + '.pcd'
	
	pcd_gt = o3d.io.read_point_cloud(inst_gt)
	#pcd_sem = o3d.io.read_point_cloud(sem_gt)
	unique_colors, labels_kitti = np.unique(np.asarray(pcd_gt.colors),axis=0, return_inverse=True)
	
	#o3d.visualization.draw_geometries([pcd_sem])
	#with np.load(chunk_folder + 'out_kitti_semantic/' + f.split('.')[0] + '.npz') as fn : 
	#	labels = fn['labels']
	
	
	pred_pcd = o3d.io.read_point_cloud(f)
	
	unique_colors, labels_ncuts = np.unique(np.asarray(pred_pcd.colors), axis=0, return_inverse=True)
	o3d.visualization.draw_geometries([pred_pcd])
	
	new_labels = remove_semantics(labels_kitti,labels_ncuts)
	colored = color_pcd_by_labels(pred_pcd,new_labels)
	o3d.visualization.draw_geometries([colored])
	metrics_pred = Metrics(name=f)
	#metrics_pred.semantic_eval(pred_pcd,pcd_sem,labels)
	'''

tarl_only7.pcd
merge_refined7.pcd
map name pcd_preprocessed/out_maps/7/merge_refined7.pcd


Processing: 100%|██████████| 455/455 [00:45<00:00,  9.93it/s]


largest idx -10


KeyboardInterrupt: 

In [6]:
pred_pcd = o3d.io.read_point_cloud('pcd_preprocessed/out_maps/5'  + '/tarl_spatial5.pcd')
o3d.visualization.draw_geometries([pred_pcd])